In [384]:
import pandas as pd
import jismesh.utils as ju
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geojson
import json
import folium
import pickle as pk

## 目标: geojson 的边界数据 转成 日本的网格数据，即计算每一个 border 包含的网格信息
![jupyter](./border_to_mesh.png)

### 原始geojson 数据读取
### 数据源 https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-N03-v2_4.html#prefecture13

In [5]:
with open("N03-20_200101.geojson") as f:
    gj = geojson.load(f)
df = pd.read_json(json.dumps(gj['features']))

In [6]:
df

,type,id,geometry,properties
0,Feature,1,"{'type': 'Polygon', 'coordinates': [[[141.2574...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00..."
1,Feature,2,"{'type': 'Polygon', 'coordinates': [[[141.3333...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00..."
2,Feature,3,"{'type': 'Polygon', 'coordinates': [[[141.375,...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00..."
3,Feature,4,"{'type': 'Polygon', 'coordinates': [[[141.3664...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00..."
4,Feature,5,"{'type': 'Polygon', 'coordinates': [[[141.3636...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00..."
...,...,...,...,...
118894,Feature,118895,"{'type': 'Polygon', 'coordinates': [[[123.0455...","{'N03_001': '沖縄県', 'N03_002': '', 'N03_003': '..."
118895,Feature,118896,"{'type': 'Polygon', 'coordinates': [[[123.0435...","{'N03_001': '沖縄県', 'N03_002': '', 'N03_003': '..."
118896,Feature,118897,"{'type': 'Polygon', 'coordinates': [[[123.0432...","{'N03_001': '沖縄県', 'N03_002': '', 'N03_003': '..."
118897,Feature,118898,"{'type': 'Polygon', 'coordinates': [[[123.0433...","{'N03_001': '沖縄県', 'N03_002': '', 'N03_003': '..."


### 对每一条数据，根据边界计算覆盖的mesh

In [177]:
def get_mesh_info(coords):
    border = Polygon(coords)
    min_lat, max_lat, min_lon, max_lon = 300.0, 0.0, 300.0, 0.0
    for coord in coords:
        lon, lat = coord[0], coord[1]
        min_lat = min(lat, min_lat)
        max_lat = max(lat, max_lat)
        min_lon = min(lon, min_lon)
        max_lon = max(lon, max_lon)
    lats = [min_lat]
    lons = [min_lon]
    while True:
        min_lat += 0.005
        lats.append(min_lat)
        if min_lat > max_lat:
            break
    while True:
        min_lon += 0.005
        lons.append(min_lon)
        if min_lon > max_lon:
            break
    
    meshes = set([])
    for lat in lats:
        for lon in lons:
            meshcode = ju.to_meshcode(lat, lon, 3)
            meshes.add(meshcode)
    qualified_mesh = []
    for meshcode in meshes:
        # 南西端の緯度経度を求める。
        lat_sw, lon_sw = ju.to_meshpoint(meshcode, 0, 0)
        # 北東端の緯度経度を求める。
        lat_ne, lon_ne = ju.to_meshpoint(meshcode, 1, 1)
        grid = Polygon([[lon_sw, lat_sw], [lon_ne, lat_sw], [lon_ne, lat_ne],[lon_sw, lat_ne],[lon_sw, lat_sw]])
        if border.intersects(grid):
            qualified_mesh.append(int(meshcode))
    return qualified_mesh
# batch 的结果存储
def batch_process(data,batch_id):
    res = []
    for border_info in data:
        border_id = border_info['id']
        coords = border_info['coords']
        mesh_codes = get_mesh_info(coords)
        res.append({"id":border_id,"meshes":mesh_codes})
    with open('output/{}.out'.format(batch_id), 'w') as f:
        json.dump(res,f)

### 原始数据切片

In [172]:
borders = df['geometry'].to_list()
borders_with_id = []
for i, border in enumerate(borders):
    borders_with_id.append({"id":i+1,"coords":border['coordinates'][0]})
total_ids = len(borders_with_id)
batch_size = int(total_ids/27)+1
start_id, end_id = 0, 0
splited_data = []
while end_id < total_ids:
    start_id = end_id 
    end_id += batch_size
    tmp = borders_with_id[start_id:end_id]
#     print(start_id, end_id)
    splited_data.append(tmp)
len(splited_data)

27

### 执行批量计算

In [173]:
# batch_process(splited_data[3], 3)
from multiprocessing import Pool
pool = Pool(processes=27)
for i, data in enumerate(splited_data):
    try:    
        pool.apply_async(batch_process,(data, i,)) 
    except Exception as e:  
        print(e) 
pool.close()
pool.join()

### 分片计算的结果合并

In [243]:
dfs = []
for i in range(27):
    df_tmp = pd.read_json("./output/{}.out".format(i), orient='records')
    dfs.append(df_tmp)
mesh_df = pd.concat(dfs,axis=0, ignore_index=True)
mesh_df

,id,meshes
0,1,"[64414210, 64414211, 64414212, 64414213, 64414..."
1,2,"[64415232, 64415233, 64415234, 64415235, 64415..."
2,3,"[64415237, 64415238, 64415239, 64415247, 64415..."
3,4,"[64414269, 64414279, 64415303, 64415304, 64415..."
4,5,"[64414209, 64414218, 64414219, 64414228, 64414..."
...,...,...
118894,118895,[36235053]
118895,118896,[36235053]
118896,118897,[36235053]
118897,118898,[36235053]


In [364]:
meshed_df = pd.merge(df, mesh_df, how='left', on=['id'])
meshed_df

,type,id,geometry,properties,meshes
0,Feature,1,"{'type': 'Polygon', 'coordinates': [[[141.2574...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00...","[64414210, 64414211, 64414212, 64414213, 64414..."
1,Feature,2,"{'type': 'Polygon', 'coordinates': [[[141.3333...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00...","[64415232, 64415233, 64415234, 64415235, 64415..."
2,Feature,3,"{'type': 'Polygon', 'coordinates': [[[141.375,...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00...","[64415237, 64415238, 64415239, 64415247, 64415..."
3,Feature,4,"{'type': 'Polygon', 'coordinates': [[[141.3664...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00...","[64414269, 64414279, 64415303, 64415304, 64415..."
4,Feature,5,"{'type': 'Polygon', 'coordinates': [[[141.3636...","{'N03_001': '北海道', 'N03_002': '石狩振興局', 'N03_00...","[64414209, 64414218, 64414219, 64414228, 64414..."
...,...,...,...,...,...
118894,Feature,118895,"{'type': 'Polygon', 'coordinates': [[[123.0455...","{'N03_001': '沖縄県', 'N03_002': '', 'N03_003': '...",[36235053]
118895,Feature,118896,"{'type': 'Polygon', 'coordinates': [[[123.0435...","{'N03_001': '沖縄県', 'N03_002': '', 'N03_003': '...",[36235053]
118896,Feature,118897,"{'type': 'Polygon', 'coordinates': [[[123.0432...","{'N03_001': '沖縄県', 'N03_002': '', 'N03_003': '...",[36235053]
118897,Feature,118898,"{'type': 'Polygon', 'coordinates': [[[123.0433...","{'N03_001': '沖縄県', 'N03_002': '', 'N03_003': '...",[36235053]


### 重新格式化成我们想要的样子

In [365]:
meshed_df['pref'] = meshed_df.apply(lambda x: x['properties']['N03_001'], axis=1)
meshed_df['county_name'] = meshed_df.apply(lambda x: x['properties']['N03_003'], axis=1)
meshed_df['city_name'] = meshed_df.apply(lambda x: x['properties']['N03_004'], axis=1)
meshed_df['city_code'] = meshed_df.apply(lambda x: x['properties']['N03_007'], axis=1)
meshed_df = meshed_df[['id','meshes','pref','county_name','city_name',"city_code","geometry"]]
meshed_df["geometry"] = meshed_df.apply(lambda x: { "type": "Feature","properties": {},"geometry":x['geometry']},axis=1)

/home/chuang/anaconda3/envs/vis20/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [392]:
meshed_df

,id,meshes,pref,county_name,city_name,city_code,geometry
0,1,"[64414210, 64414211, 64414212, 64414213, 64414...",北海道,札幌市,中央区,01101,"{'type': 'Feature', 'properties': {}, 'geometr..."
1,2,"[64415232, 64415233, 64415234, 64415235, 64415...",北海道,札幌市,北区,01102,"{'type': 'Feature', 'properties': {}, 'geometr..."
2,3,"[64415237, 64415238, 64415239, 64415247, 64415...",北海道,札幌市,東区,01103,"{'type': 'Feature', 'properties': {}, 'geometr..."
3,4,"[64414269, 64414279, 64415303, 64415304, 64415...",北海道,札幌市,白石区,01104,"{'type': 'Feature', 'properties': {}, 'geometr..."
4,5,"[64414209, 64414218, 64414219, 64414228, 64414...",北海道,札幌市,豊平区,01105,"{'type': 'Feature', 'properties': {}, 'geometr..."
...,...,...,...,...,...,...,...
118894,118895,[36235053],沖縄県,八重山郡,与那国町,47382,"{'type': 'Feature', 'properties': {}, 'geometr..."
118895,118896,[36235053],沖縄県,八重山郡,与那国町,47382,"{'type': 'Feature', 'properties': {}, 'geometr..."
118896,118897,[36235053],沖縄県,八重山郡,与那国町,47382,"{'type': 'Feature', 'properties': {}, 'geometr..."
118897,118898,[36235053],沖縄県,八重山郡,与那国町,47382,"{'type': 'Feature', 'properties': {}, 'geometr..."


In [367]:
meshed_df[meshed_df['city_code'] == ""]

,id,meshes,pref,county_name,city_name,city_code,geometry
33119,33120,"[53394706, 53394707, 53394708]",千葉県,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."
39170,39171,"[45405282, 45405283, 45405284, 45405285, 45405...",東京都,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."
39171,39172,[45405272],東京都,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."
39172,39173,[45405272],東京都,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."
39173,39174,[45405273],東京都,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."
...,...,...,...,...,...,...,...
113622,113623,[46297569],鹿児島県,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."
113623,113624,[46297569],鹿児島県,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."
113624,113625,"[46297569, 46297579]",鹿児島県,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."
113625,113626,[46297579],鹿児島県,,所属未定地,,"{'type': 'Feature', 'properties': {}, 'geometr..."


### 填充 所属未定地 的 city_code
#### 这里我们使用 县的编码 的负数来表示 该县的 所属未定地

In [368]:
def get_prefix_code_of_pref(city_codes):
    for code in city_codes:
        if code !="":
            return int(code[0:2])
tmp_df = meshed_df.groupby(['pref'])['city_code'].apply(get_prefix_code_of_pref).reset_index()
pref2city_code = tmp_df.set_index('pref').T.to_dict()
tmp_df

,pref,city_code
0,三重県,24
1,京都府,26
2,佐賀県,41
3,兵庫県,28
4,北海道,1
5,千葉県,12
6,和歌山県,30
7,埼玉県,11
8,大分県,44
9,大阪府,27


In [369]:
def assign_code_to_unknown(pref_name, city_code):
    if city_code!="":
        return city_code
    else:
        return str(-pref2city_code[pref_name]['city_code'])
meshed_df['city_code'] = meshed_df.apply(lambda x: assign_code_to_unknown(x['pref'],x['city_code']),axis=1)

/home/chuang/anaconda3/envs/vis20/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [370]:
meshed_df

,id,meshes,pref,county_name,city_name,city_code,geometry
0,1,"[64414210, 64414211, 64414212, 64414213, 64414...",北海道,札幌市,中央区,01101,"{'type': 'Feature', 'properties': {}, 'geometr..."
1,2,"[64415232, 64415233, 64415234, 64415235, 64415...",北海道,札幌市,北区,01102,"{'type': 'Feature', 'properties': {}, 'geometr..."
2,3,"[64415237, 64415238, 64415239, 64415247, 64415...",北海道,札幌市,東区,01103,"{'type': 'Feature', 'properties': {}, 'geometr..."
3,4,"[64414269, 64414279, 64415303, 64415304, 64415...",北海道,札幌市,白石区,01104,"{'type': 'Feature', 'properties': {}, 'geometr..."
4,5,"[64414209, 64414218, 64414219, 64414228, 64414...",北海道,札幌市,豊平区,01105,"{'type': 'Feature', 'properties': {}, 'geometr..."
...,...,...,...,...,...,...,...
118894,118895,[36235053],沖縄県,八重山郡,与那国町,47382,"{'type': 'Feature', 'properties': {}, 'geometr..."
118895,118896,[36235053],沖縄県,八重山郡,与那国町,47382,"{'type': 'Feature', 'properties': {}, 'geometr..."
118896,118897,[36235053],沖縄県,八重山郡,与那国町,47382,"{'type': 'Feature', 'properties': {}, 'geometr..."
118897,118898,[36235053],沖縄県,八重山郡,与那国町,47382,"{'type': 'Feature', 'properties': {}, 'geometr..."


### 数据导出(共计 388763 个网格)

In [426]:
def agg_meshes(regions):
    res = set([])
    for meshes in regions:
        res = res | set(meshes)
    return res

### 1. 按市町村

In [427]:
shichoson_level_df = meshed_df.groupby(['pref','county_name','city_name','city_code'])['meshes'].apply(agg_meshes).reset_index()
shichoson_level_df['full_name'] = shichoson_level_df['pref']+shichoson_level_df['county_name']+shichoson_level_df['city_name']
shichoson_level_df = shichoson_level_df[['city_code','full_name','meshes','pref','county_name','city_name']]
shichoson_level_df

,city_code,full_name,meshes,pref,county_name,city_name
0,24214,三重県いなべ市,"{52366336, 52366337, 52366338, 52365315, 52365...",三重県,,いなべ市
1,24210,三重県亀山市,"{52362242, 52362243, 52362244, 52362245, 52362...",三重県,,亀山市
2,24203,三重県伊勢市,"{51365571, 51365572, 51365573, 51365510, 51365...",三重県,,伊勢市
3,24216,三重県伊賀市,"{52361216, 52361220, 52361221, 52361222, 52361...",三重県,,伊賀市
4,24208,三重県名張市,"{51367110, 51367003, 51367004, 51367005, 51367...",三重県,,名張市
...,...,...,...,...,...,...
1904,46492,鹿児島県肝属郡肝付町,"{46317064, 46317065, 46317095, 46317075, 46317...",鹿児島県,肝属郡,肝付町
1905,46490,鹿児島県肝属郡錦江町,"{46305792, 46305793, 46306682, 46306683, 46306...",鹿児島県,肝属郡,錦江町
1906,46392,鹿児島県薩摩郡さつま町,"{47306324, 47307437, 47307438, 47306325, 47306...",鹿児島県,薩摩郡,さつま町
1907,46303,鹿児島県鹿児島郡三島村,"{46301215, 46301222, 46301225, 46301230, 46301...",鹿児島県,鹿児島郡,三島村


### 2.按都道府县

In [428]:
pref_level_df = meshed_df.groupby(['pref'])['meshes'].apply(agg_meshes).reset_index()
pref_level_df

,pref,meshes
0,三重県,"{52363300, 52363301, 52363302, 52363303, 52363..."
1,京都府,"{52350647, 52350657, 52350658, 52350661, 52350..."
2,佐賀県,"{50290688, 50290689, 50290695, 50290696, 50290..."
3,兵庫県,"{51347460, 51347461, 51347462, 52343403, 51347..."
4,北海道,"{67473588, 67473589, 67473590, 67473591, 67473..."
5,千葉県,"{52406038, 53395700, 53395701, 53395702, 53395..."
6,和歌山県,"{50351587, 50351589, 50351676, 50351677, 50351..."
7,埼玉県,"{53395456, 53395457, 53395458, 53395459, 53395..."
8,大分県,"{50312112, 50312115, 50312116, 50312117, 49312..."
9,大阪府,"{51356209, 51356219, 51356229, 51356239, 51356..."


### 结果校正，日本国土交通省的数据缺少了 猫岛

In [430]:
org = pref_level_df.loc[pref_level_df['pref'] == "愛媛県"]['meshes'].to_list()[0]
new = org | {50324388, 50324389}
pref_level_df.loc[pref_level_df['pref'] == "愛媛県", 'meshes'] = new

In [431]:
org = shichoson_level_df.loc[(shichoson_level_df['pref'] == "愛媛県") & (shichoson_level_df['city_name'] =="大洲市")]['meshes'].to_list()[0]
new = org | {50324388, 50324389}
shichoson_level_df.loc[(shichoson_level_df['pref'] == "愛媛県") & (shichoson_level_df['city_name'] =="大洲市"), 'meshes'] = new

In [432]:
pk.dump(shichoson_level_df,open("./shichoson_level_df.pk","wb"))
pk.dump(pref_level_df,open("./pref_level_df.pk","wb"))

## 附录

### 什么是 ”所属未定地“
#### 探索结果：所属未定地是一些小的区域，不属于任何市町村

In [285]:
def get_geojson_unknown_by_pref_name(pref_name,df):
    m = folium.Map(
        location=[35.74783480535912, 139.658258144892],
        zoom_start=10  # Limited levels of zoom for free Mapbox tiles.
    )
    unknown = df[(df['pref']==pref_name) & (df['city_name']=="所属未定地")]
    data = {"type": "FeatureCollection","features": unknown['geometry'].to_list()}
    folium.GeoJson(
            data,
            name='geojson'
    ).add_to(m)
    folium.LayerControl().add_to(m)
    return m

In [291]:
get_geojson_unknown_by_pref_name("千葉県",meshed_df)

In [401]:
# 長浜町
meshed_df[(meshed_df['pref']=="愛媛県") & (meshed_df['city_name']=="所属未定地")]
# data = {"type": "FeatureCollection","features": fuck}
# m = folium.Map(
#         location=[35.74783480535912, 139.658258144892],
#         zoom_start=10  # Limited levels of zoom for free Mapbox tiles.
# )
# folium.GeoJson(
#         data,
#         name='geojson'
# ).add_to(m)
# folium.LayerControl().add_to(m)
# m

,id,meshes,pref,county_name,city_name,city_code,geometry


In [290]:
get_geojson_unknown_by_pref_name("東京都",meshed_df)

## 如何设定 mesh搜索过程中的经纬度变化增量-> 0.005°

In [88]:
from math import cos, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a))

### 0.005 ° 的经度变化

In [89]:
d1 = distance(42.997475, 141.389636, 42.997475, 141.389636+0.005)  # 北海道
d2 = distance(0, 141.389636, 0, 141.389636+0.005) # 赤道
d1,d2

(0.40663081300788306, 0.5559746297027797)

### 0.005 ° 的纬度变化

In [90]:
d1 = distance(42.997475, 141.389636, 42.997475+0.005, 141.389636)  # 北海道
d2 = distance(0, 141.389636, 0.005, 141.389636)  # 赤道
d1, d2

(0.5559746297027797, 0.5559746297027797)